In [27]:
import numpy as np
import pandas as pd
import os
import glob

from nilm_analyzer.loaders import _EnergyDataset

In [28]:
src_path = 'seq2point/data/'
target_appliance = 'dishwasher'

appliance_splits = dict()
for file_path in glob.glob(os.path.join(src_path, target_appliance, 'house_*')):
    print(file_path)
    house = file_path.split('house_')[-1]
    train_path = os.path.join(file_path, 'train.csv')
    test_path = os.path.join(file_path, 'test.csv')
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    train_df['time'] = pd.to_datetime(train_df['time'])
    train_df.set_index('time', inplace=True)
    test_df['time'] = pd.to_datetime(test_df['time'])
    test_df.set_index('time', inplace=True)
    df = pd.concat([train_df, test_df])
    appliance_splits.update({int(house): df})

seq2point/data/dishwasher\house_1
seq2point/data/dishwasher\house_10
seq2point/data/dishwasher\house_11
seq2point/data/dishwasher\house_13
seq2point/data/dishwasher\house_15
seq2point/data/dishwasher\house_16
seq2point/data/dishwasher\house_18
seq2point/data/dishwasher\house_2
seq2point/data/dishwasher\house_20
seq2point/data/dishwasher\house_21
seq2point/data/dishwasher\house_3
seq2point/data/dishwasher\house_5
seq2point/data/dishwasher\house_6
seq2point/data/dishwasher\house_7
seq2point/data/dishwasher\house_9


In [29]:
custom_dataset = _EnergyDataset(appliance_splits)

In [30]:
custom_dataset.get_activations(threshold_x=45.0, threshold_y=80.0, min_limit=5.0, max_limit=240.0)

Extracting activations for House 1
Consumption Threshold is set to = 80.0
Time Delay Threshold is set to = 45.0 minutes
Estimating active durations of: power_dishwasher
Extracting activations for House 10
Consumption Threshold is set to = 80.0
Time Delay Threshold is set to = 45.0 minutes
Estimating active durations of: power_dishwasher
Extracting activations for House 11
Consumption Threshold is set to = 80.0
Time Delay Threshold is set to = 45.0 minutes
Estimating active durations of: power_dishwasher
Extracting activations for House 13
Consumption Threshold is set to = 80.0
Time Delay Threshold is set to = 45.0 minutes
Estimating active durations of: power_dishwasher
Extracting activations for House 15
Consumption Threshold is set to = 80.0
Time Delay Threshold is set to = 45.0 minutes
Estimating active durations of: power_dishwasher
Extracting activations for House 16
Consumption Threshold is set to = 80.0
Time Delay Threshold is set to = 45.0 minutes
Estimating active durations of

In [31]:
appliance_activations = dict()

print('\nConverting activities to arrays')
for house_number in custom_dataset.activations.keys():
    print(f'Processing for house {house_number}')
    filter_activities = custom_dataset.activations[house_number]
    activities_lst = []
    for row in range(0, len(filter_activities)):
        activities_lst.append(np.array(custom_dataset.data[house_number].loc[str(filter_activities['activity_start'].iloc[row]- pd.Timedelta(seconds=10)):
                                           str(filter_activities['activity_end'].iloc[row]+pd.Timedelta(seconds=10))][custom_dataset.data[house_number].columns[-1]]))
    appliance_activations.update({house_number:activities_lst})


Converting activities to arrays
Processing for house 1
Processing for house 10
Processing for house 11
Processing for house 13
Processing for house 15
Processing for house 16
Processing for house 18
Processing for house 2
Processing for house 20
Processing for house 21
Processing for house 3
Processing for house 5
Processing for house 6
Processing for house 7
Processing for house 9


In [32]:
import pickle
with open(f'{target_appliance}_activations.pkl', 'wb') as f:
    pickle.dump(appliance_activations, f)